This kernel based on work of [Tan, M. et al][1] approach which imply clusters generation via the minima of divergence of normalized gradient taken over CT scan and merged them with clasters generated by dot and line 3D [selective enhancement filters][2] 


  [1]: http://dx.doi.org/10.1118/1.3633941
  [2]: http://dx.doi.org/10.1118/1.1581411

In [63]:
from numpy import *
import numpy as np
import os
from scipy.ndimage import morphology 
from tqdm import tqdm
from os.path import join, basename, isfile
from scipy.ndimage import label
import scipy.ndimage.filters as filters
import scipy
from glob import glob
from multiprocessing import Pool, cpu_count
from skimage import measure


from scipy.ndimage.filters import gaussian_filter, laplace
import dicom
from scipy.linalg import norm
import pandas as pd

CPU = 24

Next five cells was taken from [Full Preprocessing Tutorial][1] by [Guido Zuidhof][2]


  [1]: https://www.kaggle.com/gzuidhof/data-science-bowl-2017/full-preprocessing-tutorial
  [2]: https://www.kaggle.com/gzuidhof

In [12]:
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = np.array([scan[0].SliceThickness] + scan[0].PixelSpacing, dtype=np.float32)

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
    
    return image, new_spacing

In [13]:
# Load the scans in given folder path
def load_scan(path):
    slices = [dicom.read_file(join(path, pslice)) 
              for pslice in glob(join(path, '*.dcm'))]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

In [14]:
def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

In [15]:
def largest_label_volume(im, bg=-1):
    vals, counts = np.unique(im, return_counts=True)

    counts = counts[vals != bg]
    vals = vals[vals != bg]

    if len(counts) > 0:
        return vals[np.argmax(counts)]
    else:
        return None

In [16]:
def segment_lung_mask(image, fill_lung_structures=True):
    
    # not actually binary, but 1 and 2. 
    # 0 is treated as background, which we do not want
    binary_image = np.array(image > -320, dtype=np.int8)+1
    labels = measure.label(binary_image)
    
    # Pick the pixel in the very corner to determine which label is air.
    #   Improvement: Pick multiple background labels from around the patient
    #   More resistant to "trays" on which the patient lays cutting the air 
    #   around the person in half
    background_label = labels[0,0,0]
    
    #Fill the air around the person
    binary_image[background_label == labels] = 2
    
    
    # Method of filling the lung structures (that is superior to something like 
    # morphological closing)
    if fill_lung_structures:
        # For every slice we determine the largest solid structure
        for i, axial_slice in enumerate(binary_image):
            axial_slice = axial_slice - 1
            labeling = measure.label(axial_slice)
            l_max = largest_label_volume(labeling, bg=0)
            
            if l_max is not None: #This slice contains some lung
                binary_image[i][labeling != l_max] = 1

    
    binary_image -= 1 #Make the image actual binary
    binary_image = 1-binary_image # Invert it, lungs are now 1
    
    # Remove other air pockets insided body
    labels = measure.label(binary_image, background=0)
    l_max = largest_label_volume(labels, bg=0)
    if l_max is not None: # There are air pockets
        binary_image[labels != l_max] = 0
 
    return binary_image

In [17]:
def load_data(path):    
    ct_scan = load_scan(path)
    patient = get_pixels_hu(ct_scan)
    patient, spacing = resample(patient, ct_scan, SPACING)
    
    mask = segment_lung_mask(patient)
    mask = morphology.binary_fill_holes(
        morphology.binary_dilation(
            morphology.binary_fill_holes(mask > 0), 
            iterations=4)
    )

    return patient, mask

In [18]:
# Constants presented in papers
SPACING = array([1., 1., 1.])
ISOLATED_THRESHOLD = -600
DOT_ENHANCED_THRESHOLD = 6
FILTERS_AMOUNT = 6
ISOLATED_MIN_VOLUME = 9
ISOLATED_MAX_VOLUME = 500
JUXTAVASCULAR_MIN_VOLUME = 9
JUXTAPLEURAL_MIN_VALUME = 1

Sigmas calculation for enhanced  nodule and vessels of different sizes   
It should be noted that the relationship  between the smoothing scales is exponential [\[here\]][1]. 


  [1]: http://link.springer.com/article/10.1007/BF00336961

In [19]:
MIN_RADIUS = 4
MAX_RADIUS = 16

def get_scales(bottom=MIN_RADIUS, top=MAX_RADIUS, 
               amount=FILTERS_AMOUNT):
    radius = (top / bottom) ** (1. / (amount - 1))
    sigmas = [bottom / 4.]
    for i in range(amount - 1):
        sigmas.append(sigmas[0] * (radius ** i + 1))
    return sigmas

Enhanced filters based on hessian propreties

In [20]:
def hessian(field, coords):
    grad = gradient(field)
    axis = [[0, 1, 2], [1, 2], [2]]
    hess = [gradient(deriv, axis=j) 
            for i, deriv in enumerate(grad) 
            for j in axis[i]]

#   [(0, xx), (1, xy), (2, xz), (3, yy), (4, yz), (5, zz)]
#   x, y, z -> 3, 3, x, y, z -> 3, 3, N

    for j in range(len(hess)):
        hess[j] = hess[j][coords]

    return asarray([[hess[0], hess[1], hess[2]],
                    [hess[1], hess[3], hess[4]],
                    [hess[2], hess[4], hess[5]]])

In [31]:
def enhanced_filter(patient, coords, sigma):
    filtered = gaussian_filter(patient, sigma=sigma)
    hess = hessian(filtered, coords=coords)
    hess = [hess[:, :, i] for i in range(hess.shape[-1])]
    with Pool(CPU) as pool:
        eigs = pool.map(linalg.eigvalsh, 
                        hess)

    sigma_sqr = sigma ** 2
    z_dot = [sigma_sqr * (eig_val[2] ** 2) / abs(eig_val[0]) 
             if eig_val[0] < 0 
             and eig_val[1] < 0 
             and eig_val[2] < 0 
             else 0
             for eig_val in eigs]

    z_line = [sigma_sqr * abs(eig_val[1]) 
              * (abs(eig_val[1]) - abs(eig_val[2])) 
              / abs(eig_val[0]) 
              if eig_val[0] < 0 
              and eig_val[1] < 0 
              else 0
              for eig_val in eigs]
    return z_dot, z_line

In [32]:
def apply_enhs_filters(patient, mask, include_plane=False):
    sigmas = get_scales()
    enh_dot = zeros(mask.shape)
    enh_line = zeros(mask.shape)
    coords = where(mask)
    
    z_dot = list()
    z_line = list()
    for sigma in sigmas:
        dot, line = enhanced_filter(patient, coords, sigma)
        z_dot.append(dot)
        z_line.append(line)


    enh_dot[coords] = asarray(z_dot).max(axis=0)
    enh_line[coords] = asarray(z_line).max(axis=0)

    return enh_dot, enh_line

Implementation of clastering via maxima divergence of normalized gradient (DNG) was shown above, but it will not going to be used further due to following two situations: 1) Why they called it maxima of DNG, instead of minima? Nodule is a concave function hence it's second derivative need to be negative. Though it may be just  a typo, by they focused attention on this.   2) Provided thresholds in a paper did not match with output values of maxima of sign changed DNG, this ofcourse may be still my mistakes, so I'll be glad if you point me on 'em.

In [23]:
def div_of_norm_grad(sigma, patient):
    grad = asarray(gradient(patient))
    grad /= norm(grad, axis=0) + 1e-3 # Smooth const
    grad = [gaussian_filter(deriv, sigma=sigma) for deriv in grad]
    return sum([gradient(el, axis=i) 
                for i, el in enumerate(grad)], axis=0)

In [24]:
def maxima_divergence(masks_pats):
    with Pool(CPU) as pool:
        divs = pool.map(
            functools.partial(divergence, 
                              patient=pat), 
            sigmas
        )
        divs = -1 * asarray(divs) * mask 
        divs = divs.max(axis=0)
        divs_list.append(divs.copy())
    return divs_list

Clastering generation was done via merging enhanced and original thresholded volumetric images

In [25]:
def is_in(colour, labe, dng_colours):
    if colour in dng_colours:
        return labe == colour


def get_pure_isol(patient, mask, enh_dot):
    isolated = (patient > -600) * (mask > 0) * (enh_dot < 6) 
    labe, iso_nodules_num = label(isolated)
    volumes = bincount(labe.flatten())
    colours = where((volumes > ISOLATED_MIN_VOLUME) 
                & (volumes < ISOLATED_MAX_VOLUME))[0]
    
    isolated = zeros(isolated.shape).astype(bool)
    for colour in colours:
        isolated |= labe == colour
        
    return isolated, iso_nodules_num


def get_pure_j_va(patient, mask, enh_line, iso):
    juxtavascular = (patient > -600) * (mask > 0) * (enh_line > 150)
    j_va_candidates = (1 - juxtavascular) * (1 - iso)
    labe, j_va_nodules_num = label(j_va_candidates)

    volumes = bincount(labe.flatten())
    colours = where((volumes > JUXTAVASCULAR_MIN_VOLUME) 
                    & (volumes < ISOLATED_MAX_VOLUME))[0]
    j_va = zeros(juxtavascular.shape).astype(bool)
    for colour in colours:
        j_va |= labe == colour
    
    return j_va, j_va_nodules_num


def get_pure_j_pl(patient, mask, enh_dot):
    fixed_mask = morphology.binary_erosion(mask > 0,iterations=4)
    border_mask = fixed_mask * (1 - morphology.binary_erosion(fixed_mask > 0,iterations=4))
    juxtapleural = (patient > -400) * (border_mask > 0) * (enh_dot > 4)

    labe, j_pl_num = label(juxtapleural)
    volumes = bincount(labe.flatten())
    colours = where((volumes > JUXTAPLEURAL_MIN_VALUME) 
                    & (volumes < ISOLATED_MAX_VOLUME))[0]
    j_pl = zeros(juxtapleural.shape).astype(bool)
    for colour in colours:
        j_pl |= labe == colour
    return j_pl, j_pl_num

In [26]:
def get_pure_nodules(patient, mask, enh):
    """
    Here: 
    1 is for isolated
    2 is for j_va
    4 is for j_pl
    """
    iso, iso_num = get_pure_isol(patient, mask, enh[0])
    j_va, j_va_num = get_pure_j_va(patient, mask, enh[1], iso)
    j_pl, j_pl_num = get_pure_j_pl(patient, mask, enh[0])
    return 2 * j_va + iso + 4 * j_pl, (iso_num, j_va_num, j_pl_num)

It tooks around 500s  and resulted on average 730 candidates  per patient. 

In [64]:
def operate(path):
    patient, mask = load_data(path)
    enhs = apply_enhs_filters(patient, mask, 
                              include_plane=False)
    nodules, amounts = get_pure_nodules(patient, mask, enhs)

Due to the Kaggle kernel limitation I post the results in this form with candidates in center of the images:
 ![Due to the Kaggle kernel limitation I post the results in this form][1]


  [1]: https://s30.postimg.org/6lnslbdox/index.png